<a href="https://colab.research.google.com/github/alinapradhan/Kuzushiji-MNIST-CNN/blob/main/CNNJapanese_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import requests
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


# 1. Download Kuzushiji-MNIST (.npz)
base_url = "http://codh.rois.ac.jp/kmnist/dataset/kmnist/"
files = [
    "kmnist-train-imgs.npz",
    "kmnist-train-labels.npz",
    "kmnist-test-imgs.npz",
    "kmnist-test-labels.npz"
]

os.makedirs("kmnist_data", exist_ok=True)

for fname in files:
    path = os.path.join("kmnist_data", fname)
    if not os.path.exists(path):
        print(f"Downloading {fname}...")
        url = base_url + fname
        r = requests.get(url)
        with open(path, "wb") as f:
            f.write(r.content)
    else:
        print(f"{fname} already exists. Skipping download.")


# 2. Load data
train_images = np.load("kmnist_data/kmnist-train-imgs.npz")['arr_0']
train_labels = np.load("kmnist_data/kmnist-train-labels.npz")['arr_0']
test_images = np.load("kmnist_data/kmnist-test-imgs.npz")['arr_0']
test_labels = np.load("kmnist_data/kmnist-test-labels.npz")['arr_0']

# 3. Preprocess
train_images = train_images.reshape(-1, 28, 28, 1).astype("float32") / 255.0
test_images = test_images.reshape(-1, 28, 28, 1).astype("float32") / 255.0

train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)


# 4. Define CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# 5. Train model
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(train_images, train_labels,
                    validation_split=0.2,
                    epochs=20,
                    batch_size=128,
                    callbacks=[early_stop],
                    verbose=1)


# 6. Evaluate
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")


# 7. Save model
model.save("kuzushiji_mnist_cnn.h5")
print("Model saved as kuzushiji_mnist_cnn.h5")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 40s 102ms/step - accuracy: 0.7187 - loss: 0.8832 - val_accuracy: 0.9458 - val_loss: 0.1858
Epoch 2/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 38s 101ms/step - accuracy: 0.9378 - loss: 0.2028 - val_accuracy: 0.9668 - val_loss: 0.1182
Epoch 3/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 39s 95ms/step - accuracy: 0.9574 - loss: 0.1414 - val_accuracy: 0.9731 - val_loss: 0.0908
Epoch 4/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 43s 100ms/step - accuracy: 0.9691 - loss: 0.0979 - val_accuracy: 0.9777 - val_loss: 0.0730
Epoch 5/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 40s 97ms/step - accuracy: 0.9745 - loss: 0.0821 - val_accuracy: 0.9807 - val_loss: 0.0668
Epoch 6/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 36s 96ms/step - accuracy: 0.9801 - loss: 0.0635 - val_accuracy: 0.9807 - val_loss: 0.0638
Epoch 7/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 41s 96ms/step - accuracy: 0.9811 - loss: 0.0590 - val_accuracy: 0.9817 - val_loss: 0.0613
Epoch 8/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 36s 96ms/step - accuracy: 0.9846 - loss: 0.0471

Test Accuracy: 0.9523
Model saved as kuzushiji_mnist_cnn.h5
